In [ ]:
'''
Unit 7.6 Data Wrangling for Capstone 2
Nantawat Samermit 
'''

In [6]:
#import relevant modules for data wrangling - keep on adding as needed
import pandas as pd
import json, requests
import fuzzywuzzy 
import matplotlib

In [4]:
#ask Luka: Should I just use requests or should I use the API given by the data set manager?
#here is a code snippet from the API doc
#the source is: https://dev.socrata.com/foundry/data.cambridgema.gov/ayt4-g2ye

#from sodapy import Socrates
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cambridgema.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cambridgema.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
#results = client.get("ayt4-g2ye", limit=2000)

# Convert to pandas DataFrame
#results_df = pd.DataFrame.from_records(results)

In [5]:
#Cambridge MA wastewater dataset as JSON
url = 'https://data.cambridgema.gov/resource/ayt4-g2ye.json'

In [6]:
request = requests.get(url)
json_data = request.json()
json_data[0]

{'date': '2020-10-01T00:00:00.000',
 'mwra_concentration': '',
 'mwra_7dayaverage': '31',
 'concentration_mid_cambridge': '',
 'concentration_east_cambridge': '',
 'concentration_cambridgeport': '',
 'concentration_north_west': '',
 'highci_mid_cambridge': '',
 'highci_east_cambridge': '',
 'highci_cambridgeport': '',
 'highci_north_west_cambridge': '',
 'lowci_mid_cambridge': '',
 'lowci_east_cambridge': '',
 'lowci_cambridgeport': '',
 'lowci_north_west_cambridge': ''}

In [6]:
#confirming json_data is NOT a df yet
type(json_data)

list

In [7]:
#converting json_data to df
Cambridge_df = pd.DataFrame(json_data)

In [8]:
type(Cambridge_df)

pandas.core.frame.DataFrame

In [9]:
Cambridge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   date                          253 non-null    object
 1   mwra_concentration            253 non-null    object
 2   mwra_7dayaverage              253 non-null    object
 3   concentration_mid_cambridge   253 non-null    object
 4   concentration_east_cambridge  253 non-null    object
 5   concentration_cambridgeport   253 non-null    object
 6   concentration_north_west      253 non-null    object
 7   highci_mid_cambridge          253 non-null    object
 8   highci_east_cambridge         253 non-null    object
 9   highci_cambridgeport          253 non-null    object
 10  highci_north_west_cambridge   253 non-null    object
 11  lowci_mid_cambridge           253 non-null    object
 12  lowci_east_cambridge          253 non-null    object
 13  lowci_cambridgeport 

In [10]:
Cambridge_df.describe()

,date,mwra_concentration,mwra_7dayaverage,concentration_mid_cambridge,concentration_east_cambridge,concentration_cambridgeport,concentration_north_west,highci_mid_cambridge,highci_east_cambridge,highci_cambridgeport,highci_north_west_cambridge,lowci_mid_cambridge,lowci_east_cambridge,lowci_cambridgeport,lowci_north_west_cambridge
count,253,253,253,253,253,253,253,253,253,253,253,253,253,253,253
unique,253,216,197,28,30,24,30,28,30,24,30,28,30,24,30
top,2021-01-05T00:00:00.000,,257,,,,,,,,,,,,
freq,1,10,5,226,224,228,224,226,224,228,224,226,224,228,224


In [11]:
'''
For my own review!

Column Definitions:
Date - sample date for 24 hour composite sample
MWRA_Concentration - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
MWRA_7DayAverage - 7 day rolling average for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
Concentration_* - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
highci_* - Upper Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
lowci_* - Lower Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.
* means there are multiple columns with this prefix which share the same definition - Nantawat

A refresher on confidence interval limits - https://bit.ly/3vtuI7h
'''

'\nFor my own review!\n\nColumn Definitions:\nDate - sample date for 24 hour composite sample\nMWRA_Concentration - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nMWRA_7DayAverage - 7 day rolling average for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nConcentration_* - Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nhighci_* - Upper Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\nlowci_* - Lower Limit on Confidence Interval for Normalized SARS-CoV2 RNA Copies per ML collected during sampling period.\n* means there are multiple columns with this prefix which share the same definition - Nantawat\n\nA refresher on confidence interval limits - https://bit.ly/3vtuI7h\n'

In [12]:
Cambridge_df.tail()

,date,mwra_concentration,mwra_7dayaverage,concentration_mid_cambridge,concentration_east_cambridge,concentration_cambridgeport,concentration_north_west,highci_mid_cambridge,highci_east_cambridge,highci_cambridgeport,highci_north_west_cambridge,lowci_mid_cambridge,lowci_east_cambridge,lowci_cambridgeport,lowci_north_west_cambridge
248,2021-06-06T00:00:00.000,24,25,,,,,,,,,,,,
249,2021-06-07T00:00:00.000,29,27,,,,,,,,,,,,
250,2021-06-08T00:00:00.000,15,24,,,,,,,,,,,,
251,2021-06-09T00:00:00.000,14,22,,,,,,,,,,,,
252,2021-06-10T00:00:00.000,33,23,,,,,,,,,,,,


In [13]:
#Note: There seems to be plenty of empty values, must make sure to sanitize them to NaN 
# The last date entry is for 6/10/21

In [7]:
#scraping for Massachusett's DoH dataset. Note this dataset's last entry is for 6/15/21
url2 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download'

In [8]:
#This seems like a big file, can I stream it in chunks?
Mass_request = requests.get(url2)    



In [26]:
#Note: Mass_request.content reveals something funky, can't describe it
#Mass_request.content
Mass_request.headers

{'Date': 'Wed, 16 Jun 2021 18:35:42 GMT', 'Content-Type': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'Content-Length': '2482381', 'Connection': 'keep-alive', 'CF-Ray': '660623b5dfc9286f-DFW', 'Accept-Ranges': 'bytes', 'Age': '75526', 'Cache-Control': 'public, max-age=60', 'Content-Disposition': 'inline; filename="covid-19-raw-data-file-6-15-2021.xlsx"', 'Expires': 'Wed, 30 Jun 2021 18:35:42 GMT', 'Last-Modified': 'Tue, 15 Jun 2021 21:06:44 GMT', 'Vary': 'Accept-Encoding', 'Via': 'varnish', 'CF-Cache-Status': 'HIT', 'cf-request-id': '0ab7b4a5a90000286f81293000000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'X-Cache': 'HIT', 'X-Cache-Hits': '12', 'X-Content-Type-Options': 'nosniff', 'X-Request-ID': 'v-aa5a8f2e-ce1d-11eb-8214-5774f8677340', 'Set-Cookie': '__cfruid=c33e3f8bec17822f86d4e99d11cd5f8e1758c127-1623868542; path=/; domain=.mass.gov; HttpOnly; Secure; SameSite=None', 'Server': 'cloudflare', 

In [9]:
#from this StackOverflow article, BytesIO might be able to help
# https://stackoverflow.com/questions/58147689/python-how-to-read-excel-file-from-requests-response
import io

In [10]:
with io.BytesIO(Mass_request.content) as file:
    df = pd.io.excel.read_excel(file)

In [14]:
df.columns

Index(['Current Table Name', 'Old Table Name', 'Current Variable Name',
       'Old Variable Name', 'Frequency of Data Updates', 'Description',
       'Notes', 'Unnamed: 7'],
      dtype='object')

In [2]:
#Note: df does not match the excel spreadsheet I downloaded, something went wrong. 
#Maybe the url2 request didn't grab the .xls
#Checked in my chrome downloads folder, the .xls should be around 2,425 kb. Small enough for bs4?

In [15]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

In [63]:
#Request is forbidden 403 - try BS4
#url2 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download'
#request = Request(url2)
#response = urlopen(request)
#print(type(response))
#response.close()

HTTPError: HTTP Error 403: Forbidden

In [64]:
#Once again, the file is too big! Must consider streaming in chunks
#r = requests.get(url2)
#text = r.text
#print(text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
#found a link to the specific xls
#testing to see if BS4 can ID and isolate the href with .xls 
url3 = 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021'
r = requests.get(url3)
html_doc = r.text
soup = BeautifulSoup(html_doc)
print(soup.title)

<title>covid-19-raw-data-file-6-15-2021.xlsx | Mass.gov</title>


In [20]:
file_format = '.xlsx'
a_tags = soup.find_all('a')

for link in a_tags:
    print(link.get('href'))

#main-content
/info-details/massachusetts-state-organizations-a-to-z
https://sso.hhs.state.ma.us/
https://uionline.detma.org/Claimant/Core/Login.ASPX
https://ecse.cse.state.ma.us/ECSE/Login/login.asp
/
/topics/health-social-services
/topics/families-children
/topics/housing-property
/topics/transportation
/topics/legal-justice
/topics/public-safety
/topics/energy
/topics/environment
/topics/taxes
/topics/unclaimed-property
/topics/vital-public-records
/topics/voting
/topics/living
/topics/business-resources
/topics/professional-licenses-permits
/topics/unemployment
/topics/finding-a-job
/topics/taxes
/topics/workers-rights-safety
/topics/industry-regulations
/topics/state-employee-resources
/topics/working
/topics/early-childhood-education-care
/topics/elementary-secondary-schools
/topics/higher-education
/topics/continuing-education
/topics/for-educators-administrators
/topics/professional-training-career-development
/topics/learning
/topics/parks-recreation
/topics/fishing-hunting
/t

In [ ]:
#Stuck here. It seems the xls href doesn't point to the file itself, but maybe to a webapp /download
#which obfuscates the file location. Ask Luka

In [30]:
#Notes from call with Luka 6.16.21
# Use BS to identify/isolate the html "class" pointing to the href of the .xls 
#class and "ma__download-link__file-link" was identified by inspecting the page's elements in chrome
source = requests.get("https://www.mass.gov/doc/covid-19-raw-data-june-15-2021")
BeautifulSoup(source.content).find("a", {"class":"ma__download-link__file-link"}).get("href")

'/doc/covid-19-raw-data-june-15-2021/download'

In [60]:
#BS isolated the download link as: '/doc/covid-19-raw-data-june-15-2021/download'
Mass_request = requests.get("https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download", allow_redirects=True)

#Using a context manager to create a .xls file and write content from the Mass_request Response object
with open("Mass_DoH_data.xlsx", "wb") as file:
    file.write(Mass_request.content)
 
# open('Mass_DoH_data.xlsx', 'wb').write(Mass_request.content)
    
#convert to pd DataFrame using pd.read_excel() - this is for testing - should implement chunk streaming
Mass_DoH_df = pd.read_excel("Mass_DoH_data.xlsx", sheet_name=None)

In [61]:
# As the downloaded file has multiple sheets, the sheets has been stored as a dict
# where the dict is {sheetname: DataFrame}
# Confirm that sheetname stored as keys
for key in Mass_DoH_df.keys():
    print(key)

Data Documentation
Weekly_Town_Reference
Age Means Last2Weeks
AgeLast2Weeks
Cases (Report Date)
CasesbyAge
CasesByDate (Test Date)
CasesByDate_Probable
County_Daily
County_Weekly
CountyDeaths
DateofDeath
DeathsReported (Report Date)
DeathCharacteristics
HigherEd_CasesandTests
HospBedAvailable-Regional
HospBed-Hospital COVID Census
Hospitalization from Hospitals
LTC Facilities
RaceEthnicityLast2Weeks
SexLast2Weeks
Testing2 (Report Date)
TestingByDate (Test Date)
TestingPosByAge
Weekly_City_Town
Weekly_Statewide
Clusters
Isolation and Quarantine
Contact Tracing
CTC workforce
Counts by Specimen Date (Sero)


In [56]:
Mass_DoH_df['CasesByDate (Test Date)'].shape

(503, 4)

In [59]:
#confirm that Mass_DoH_df[sheetname] stored as DF
type(Mass_DoH_df['CasesByDate (Test Date)'])

pandas.core.frame.DataFrame

In [175]:
#Luka states it is easier (and acceptable) to just download the files to local path. 

#For the mass.gov dataset, it has mulitiple sheets, so it isolates the intended sheet. (not dynamic, ikr)
sources = ['https://data.cambridgema.gov/resource/ayt4-g2ye.json', 'https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download']

def get_sources(source_list):
    '''
    this function takes a list of urls, ID's their format, requests dataset from source,
    converts data into a dataframe, then saves it as a pickled file.
    '''
    for source in source_list:
        if source.endswith('.json'):
            response = requests.get(source)
            data = response.json()
            json_df = pd.DataFrame(data)
            json_df.to_pickle('Cambridge.pkl')
            print('Cambrdige.pkl created using this source: ', source)
            
        else:
            response = requests.get(source, allow_redirects=True)
            with open('excel_data.xlsx', 'wb') as file:
                file.write(response.content)
            
            #using pd.read_excel on a multi-spreadsheet xls yields a dictionary
            excel_dict = pd.read_excel('excel_data.xlsx', sheet_name=None)
            
            #can ID sheetnames by evaluating keys
            #print(excel_dict.keys())
            
            #Will be using the "Weekly_City_Town" sheet
            Mass_weekly_city = excel_dict['Weekly_City_Town']
            
            #Use df.to_pickle to save as .pkl file
            Mass_weekly_city.to_pickle('Mass_Weekly_City_df.pkl')
            
            print('Mass_Weekly_City.pkl created using this source', source)

In [176]:
get_sources(sources)

Cambrdige.pkl created using this source:  https://data.cambridgema.gov/resource/ayt4-g2ye.json
Mass_Weekly_City.pkl created using this source https://www.mass.gov/doc/covid-19-raw-data-june-15-2021/download


In [16]:
#confirm I can load pickled df's
import pickle

#should use context manager to load pickles
with open('Cambridge.pkl', 'rb') as f:
    Cambridge_wastewater = pickle.load(f)


In [17]:
Cambridge_wastewater.columns

Index(['date', 'mwra_concentration', 'mwra_7dayaverage',
       'concentration_mid_cambridge', 'concentration_east_cambridge',
       'concentration_cambridgeport', 'concentration_north_west',
       'highci_mid_cambridge', 'highci_east_cambridge', 'highci_cambridgeport',
       'highci_north_west_cambridge', 'lowci_mid_cambridge',
       'lowci_east_cambridge', 'lowci_cambridgeport',
       'lowci_north_west_cambridge'],
      dtype='object')

In [18]:
#load Mass_DoH pickle
with open('Mass_Weekly_City_df.pkl', 'rb') as f:
    Mass_Weekly_City_df = pickle.load(f)

In [19]:
Mass_Weekly_City_df.shape

(8800, 17)

In [20]:
Mass_Weekly_City_df.columns

Index(['City/Town', 'County', 'Population', 'Total Case Counts',
       'Two Week Case Counts', 'Average Daily Rate', 'Color',
       'Change in Last Week', 'Total Tests', 'Total Tests Last Two Weeks',
       'Total Positive Tests', 'Percent Positivity', 'Change Since Last Week',
       'Testing Rate', 'Report Date', 'Start_Date', 'End_Date'],
      dtype='object')

In [21]:
Cambridge_weekly_trend = Mass_Weekly_City_df[Mass_Weekly_City_df['City/Town'] == 'Cambridge']

In [22]:
Cambridge_weekly_trend.shape

(25, 17)

In [23]:
Cambridge_weekly_trend.index

Int64Index([  49,  401,  753, 1105, 1457, 1809, 2161, 2513, 2865, 3217, 3569,
            3921, 4273, 4625, 4977, 5329, 5681, 6033, 6385, 6737, 7089, 7441,
            7793, 8145, 8497],
           dtype='int64')

In [24]:
#dropping index of Cambridge_weekly_trend
#inplace=False will allow a copy to be assigned to Cambridge_weekly - Nantawat
Cambridge_weekly = Cambridge_weekly_trend.reset_index(drop=True, inplace=False)

In [25]:
Cambridge_weekly.index

RangeIndex(start=0, stop=25, step=1)

In [26]:
#Time to clean up Cambridge_weekly_trend
#Because I isolated Cambridge when defining Cambridge_weekly_trend, I can drop the 'City/Town' column
#But before I do, I should check if there are any null City/Town entries
Cambridge_weekly['City/Town'].isnull().values.any()

False

In [27]:
#check to see if any entries in 'City/Town' does not match with 'Cambridge'
Cambridge_weekly['City/Town'].unique()

array(['Cambridge'], dtype=object)

In [28]:
#I feel safe in dropping city/town. Check for County
Cambridge_weekly['County'].isnull().values.any()

False

In [29]:
Cambridge_weekly['County'].unique()

array(['Middlesex'], dtype=object)

In [30]:
#I feel safe in dropping County
#pre-drop checking of columns
Cambridge_weekly.columns

Index(['City/Town', 'County', 'Population', 'Total Case Counts',
       'Two Week Case Counts', 'Average Daily Rate', 'Color',
       'Change in Last Week', 'Total Tests', 'Total Tests Last Two Weeks',
       'Total Positive Tests', 'Percent Positivity', 'Change Since Last Week',
       'Testing Rate', 'Report Date', 'Start_Date', 'End_Date'],
      dtype='object')

In [31]:
labels = ['City/Town', 'County']
weekly_trends = Cambridge_weekly.drop(columns = labels)

In [32]:
#confirmed the two columns were dropped
weekly_trends.columns

Index(['Population', 'Total Case Counts', 'Two Week Case Counts',
       'Average Daily Rate', 'Color', 'Change in Last Week', 'Total Tests',
       'Total Tests Last Two Weeks', 'Total Positive Tests',
       'Percent Positivity', 'Change Since Last Week', 'Testing Rate',
       'Report Date', 'Start_Date', 'End_Date'],
      dtype='object')

In [33]:
#Exploring data type of 'Report Date' column, reviewing for null values, may need to cast as pd.datetime
weekly_trends['Report Date'].shape
weekly_trends['Report Date'].describe
weekly_trends['Report Date'].head

<bound method NDFrame.head of 0    2020-12-24
1    2020-12-31
2    2021-01-07
3    2021-01-14
4    2021-01-21
5    2021-01-28
6    2021-02-04
7    2021-02-11
8    2021-02-18
9    2021-02-25
10   2021-03-04
11   2021-03-11
12   2021-03-18
13   2021-03-25
14   2021-04-01
15   2021-04-08
16   2021-04-15
17   2021-04-22
18   2021-04-29
19   2021-05-06
20   2021-05-13
21   2021-05-20
22   2021-05-27
23   2021-06-03
24   2021-06-10
Name: Report Date, dtype: datetime64[ns]>

In [34]:
#Cambridge wastewater surveillance data starts at 10/01/2020, will select this as the minimum date range
#something funky, there is an index for this column? - fixed this with reset_index for Cambridge_weekly - Nantawat
weekly_trends['Report Date'].index

RangeIndex(start=0, stop=25, step=1)

In [35]:
#explicitly convert to datetime object by the 'infer_datetime_format = 'coerce' flag
weekly_trends['dt'] = pd.to_datetime(weekly_trends['Report Date'], 
                                     infer_datetime_format='coerce')
weekly_trends['dt'].describe

<bound method NDFrame.describe of 0    2020-12-24
1    2020-12-31
2    2021-01-07
3    2021-01-14
4    2021-01-21
5    2021-01-28
6    2021-02-04
7    2021-02-11
8    2021-02-18
9    2021-02-25
10   2021-03-04
11   2021-03-11
12   2021-03-18
13   2021-03-25
14   2021-04-01
15   2021-04-08
16   2021-04-15
17   2021-04-22
18   2021-04-29
19   2021-05-06
20   2021-05-13
21   2021-05-20
22   2021-05-27
23   2021-06-03
24   2021-06-10
Name: dt, dtype: datetime64[ns]>

In [36]:
#should convert 'Report Date' to match with Cambridge_wastewater
Cambridge_wastewater.columns

Index(['date', 'mwra_concentration', 'mwra_7dayaverage',
       'concentration_mid_cambridge', 'concentration_east_cambridge',
       'concentration_cambridgeport', 'concentration_north_west',
       'highci_mid_cambridge', 'highci_east_cambridge', 'highci_cambridgeport',
       'highci_north_west_cambridge', 'lowci_mid_cambridge',
       'lowci_east_cambridge', 'lowci_cambridgeport',
       'lowci_north_west_cambridge'],
      dtype='object')

In [37]:
#shows wastewater['date']'s formatting is inconsistent with weekly_trends
Cambridge_wastewater['date'].describe

<bound method NDFrame.describe of 0      2020-10-01T00:00:00.000
1      2020-10-02T00:00:00.000
2      2020-10-03T00:00:00.000
3      2020-10-04T00:00:00.000
4      2020-10-05T00:00:00.000
                ...           
248    2021-06-06T00:00:00.000
249    2021-06-07T00:00:00.000
250    2021-06-08T00:00:00.000
251    2021-06-09T00:00:00.000
252    2021-06-10T00:00:00.000
Name: date, Length: 253, dtype: object>

In [38]:
#confirming no null values
Cambridge_wastewater['date'].isnull().values.any()

False

In [39]:
#casting the entire 'date' column in wastewater to datetime type, will match weekly_trends
Cambridge_wastewater['dt'] = pd.to_datetime(Cambridge_wastewater['date'])
Cambridge_wastewater['dt'].describe

<bound method NDFrame.describe of 0     2020-10-01
1     2020-10-02
2     2020-10-03
3     2020-10-04
4     2020-10-05
         ...    
248   2021-06-06
249   2021-06-07
250   2021-06-08
251   2021-06-09
252   2021-06-10
Name: dt, Length: 253, dtype: datetime64[ns]>

In [40]:
#now both wastewater and weekly_trends datetime types matches
type(Cambridge_wastewater['dt']) == type(weekly_trends['dt'])

True

In [44]:
#check for any duplicate values
wastewater_duplicated_list = list(Cambridge_wastewater.duplicated(subset='dt'))

#tests if there are any True values from the list populated by .duplicated()
#confirms no True - no duplicated values in Cambridge_wastewater. 
#casted .duplicated into list because there were too many rows to visually inspect
True in wastewater_duplicated_list

False

In [42]:
#small enough output to visually confirm no duplicated values in weekly_trends
weekly_trends.duplicated(subset='dt')

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
dtype: bool

In [ ]:
# From this point, can unify the two df's with a join based on their respective 'dt' columns as index
# I don't think it is ready to join yet, will have to check other columns of interest for missing values
# ID data types (categorical vs. numerical). Pause here - NS


In [ ]:
#resume work 6.17.21
#let's work on Data Definition
#ask: 
#Do column names correspond to what those columns store?
#Are data types within columns sensible?
#Calculate summary statistics - mean, median, mode, std. dev., range, and number of unique values

In [69]:
weekly_columns = weekly_trends.columns
wastewater_columns = Cambridge_wastewater.columns
print(weekly_columns, wastewater_columns)

Index(['Population', 'Total Case Counts', 'Two Week Case Counts',
       'Average Daily Rate', 'Color', 'Change in Last Week', 'Total Tests',
       'Total Tests Last Two Weeks', 'Total Positive Tests',
       'Percent Positivity', 'Change Since Last Week', 'Testing Rate',
       'Report Date', 'Start_Date', 'End_Date', 'dt'],
      dtype='object') Index(['date', 'mwra_concentration', 'mwra_7dayaverage',
       'concentration_mid_cambridge', 'concentration_east_cambridge',
       'concentration_cambridgeport', 'concentration_north_west',
       'highci_mid_cambridge', 'highci_east_cambridge', 'highci_cambridgeport',
       'highci_north_west_cambridge', 'lowci_mid_cambridge',
       'lowci_east_cambridge', 'lowci_cambridgeport',
       'lowci_north_west_cambridge', 'dt'],
      dtype='object')


In [71]:
#categorical datatype in Change in Last week
change_in_last_week = pd.DataFrame(weekly_trends['Change in Last Week']).set_index(keys=weekly_trends.dt, 
                                                                     inplace=False)
print(change_in_last_week)

           Change in Last Week
dt                            
2020-12-24               Lower
2020-12-31               Lower
2021-01-07              Higher
2021-01-14              Higher
2021-01-21              Higher
2021-01-28               Lower
2021-02-04               Lower
2021-02-11               Lower
2021-02-18               Lower
2021-02-25               Lower
2021-03-04              Higher
2021-03-11              Higher
2021-03-18               Lower
2021-03-25               Lower
2021-04-01              Higher
2021-04-08              Higher
2021-04-15               Lower
2021-04-22               Lower
2021-04-29               Lower
2021-05-06               Lower
2021-05-13               Lower
2021-05-20               Lower
2021-05-27               Lower
2021-06-03               Lower
2021-06-10               Lower


In [73]:
#categorical datatype in Change Since Last week
#this might be a better metric to use when looking and surge prediction - Nantawat
change_since_last_week = pd.DataFrame(weekly_trends['Change Since Last Week']).set_index(keys=weekly_trends.dt, 
                                                                     inplace=False)
print(change_since_last_week)

           Change Since Last Week
dt                               
2020-12-24              No Change
2020-12-31                 Higher
2021-01-07                 Higher
2021-01-14              No Change
2021-01-21                  Lower
2021-01-28                  Lower
2021-02-04                  Lower
2021-02-11                  Lower
2021-02-18                  Lower
2021-02-25                  Lower
2021-03-04              No Change
2021-03-11              No Change
2021-03-18              No Change
2021-03-25              No Change
2021-04-01                 Higher
2021-04-08              No Change
2021-04-15              No Change
2021-04-22              No Change
2021-04-29                  Lower
2021-05-06                  Lower
2021-05-13                  Lower
2021-05-20              No Change
2021-05-27              No Change
2021-06-03              No Change
2021-06-10              No Change


In [46]:
weekly_trends['Average Daily Rate'].describe

<bound method NDFrame.describe of 0     26.852121
1     26.022958
2      26.78834
3     32.847607
4     35.335095
5     32.337353
6     27.553721
7     21.494453
8     14.542241
9     12.756352
10    13.840642
11    16.710821
12    16.328131
13    15.180059
14    20.473945
15    22.514962
16    21.494453
17    20.027473
18    12.820134
19     9.120792
20     6.378176
21     2.997743
22     2.423707
23     1.913453
24     1.722108
Name: Average Daily Rate, dtype: object>

In [84]:
#i'd like to see this paired with the 'dt' as index
Average_Daily = pd.DataFrame(weekly_trends['Average Daily Rate'].astype(float)).set_index(
    keys = weekly_trends.dt, inplace = False)

In [85]:
Average_Daily.describe

<bound method NDFrame.describe of             Average Daily Rate
dt                            
2020-12-24           26.852121
2020-12-31           26.022958
2021-01-07           26.788340
2021-01-14           32.847607
2021-01-21           35.335095
2021-01-28           32.337353
2021-02-04           27.553721
2021-02-11           21.494453
2021-02-18           14.542241
2021-02-25           12.756352
2021-03-04           13.840642
2021-03-11           16.710821
2021-03-18           16.328131
2021-03-25           15.180059
2021-04-01           20.473945
2021-04-08           22.514962
2021-04-15           21.494453
2021-04-22           20.027473
2021-04-29           12.820134
2021-05-06            9.120792
2021-05-13            6.378176
2021-05-20            2.997743
2021-05-27            2.423707
2021-06-03            1.913453
2021-06-10            1.722108>

In [87]:
Average_daily_mean = Average_Daily.mean()
Average_daily_median = Average_Daily.median()
Average_daily_mode = Average_Daily.mode()
print(Average_daily_mode)

    Average Daily Rate
0             1.722108
1             1.913453
2             2.423707
3             2.997743
4             6.378176
5             9.120792
6            12.756352
7            12.820134
8            13.840642
9            14.542241
10           15.180059
11           16.328131
12           16.710821
13           20.027473
14           20.473945
15           21.494453
16           21.494453
17           22.514962
18           26.022958
19           26.788340
20           26.852121
21           27.553721
22           32.337353
23           32.847607
24           35.335095


In [56]:
weekly_trends.set_index(weekly_trends.dt, inplace = True)
print(weekly_trends.head)

<bound method NDFrame.head of                Population Total Case Counts Two Week Case Counts  \
dt                                                                 
2020-12-24  111989.024088              2645                  421   
2020-12-31  111989.024088              2835                  408   
2021-01-07  111989.024088              3083                  420   
2021-01-14  111989.024088              3346                  515   
2021-01-21  111989.024088              3618                  554   
2021-01-28  111989.024088              3862                  507   
2021-02-04  111989.024088              4058                  432   
2021-02-11  111989.024088              4177                  337   
2021-02-18  111989.024088              4271                  228   
2021-02-25  111989.024088              4377                  200   
2021-03-04  111989.024088              4486                  217   
2021-03-11  111989.024088              4623                  262   
2021-03-18  111989

In [ ]:
#ask Luka to review 6.17.21 NS